In [6]:
import pandas as pd

# load dataset
url = "https://raw.githubusercontent.com/fivethirtyeight/data/master/candy-power-ranking/candy-data.csv"
df = pd.read_csv(url, index_col=0)

# saved data also in folder to look at

# show data info
print(df.head())
print(df.info())

                chocolate  fruity  caramel  peanutyalmondy  nougat  \
competitorname                                                       
100 Grand               1       0        1               0       0   
3 Musketeers            1       0        0               0       1   
One dime                0       0        0               0       0   
One quarter             0       0        0               0       0   
Air Heads               0       1        0               0       0   

                crispedricewafer  hard  bar  pluribus  sugarpercent  \
competitorname                                                        
100 Grand                      1     0    1         0         0.732   
3 Musketeers                   0     0    1         0         0.604   
One dime                       0     0    0         0         0.011   
One quarter                    0     0    0         0         0.011   
Air Heads                      0     0    0         0         0.906   

           

In [8]:
# Korrelationsmatrix berechnen
correlation_matrix = df.corr()

# Korrelationen mit der Beliebtheitsspalte sortieren
popularities = correlation_matrix['winpercent'].sort_values(ascending=False)

print(popularities)

winpercent          1.000000
chocolate           0.636517
bar                 0.429929
peanutyalmondy      0.406192
pricepercent        0.345325
crispedricewafer    0.324680
sugarpercent        0.229151
caramel             0.213416
nougat              0.199375
pluribus           -0.247448
hard               -0.310382
fruity             -0.380938
Name: winpercent, dtype: float64
